# 특허·실용 공개·등록공보(DART에 없는 기업도 포함)
- applicantName과 company_name 일치하게
- 앞뒤 공백 제거, 모든 공백 제거, 대소문자 통일

In [ ]:
import os
import requests
import xmltodict
import pandas as pd
from dotenv import load_dotenv
import re

# =========================================================
# 1. 기본 설정
# =========================================================
load_dotenv()

API_KEY = os.getenv("KIPRIS_API_KEY")
if not API_KEY:
    raise RuntimeError("KIPRIS_API_KEY가 설정되어 있지 않습니다.")

BASE_URL = (
    "http://plus.kipris.or.kr/kipo-api/"
    "kipi/patUtiModInfoSearchSevice/getAdvancedSearch"
)

# =========================================================
# 2. 모든 기업
# =========================================================
company_df = pd.read_csv('./data/특허추출_기업명세.csv', encoding='cp949')
company_df.head()


In [ ]:
print(len(company_df))

In [ ]:
# =========================================================
# 3. 기업명 정규화 함수
#    - 공백 제거
#    - (주), 주식회사, ㈜ 제거
#    - 대소문자 통일
# =========================================================
import re
import pandas as pd

def normalize_name_strict(name):
    if pd.isna(name):
        return ""

    name = name.lower()
    name = name.strip()
    name = re.sub(r"\(주\)|㈜|주식회사", "", name)
    name = re.sub(r"\s+", "", name)

    return name

# =========================================================
# 4. 기업 1곳 특허 정보 수집 함수 (KIPRIS)
# =========================================================
def fetch_patent_by_company(company_name, max_pages=10, num_rows=50):
    all_items = []

    for page_no in range(1, max_pages + 1):
        params = {
            "ServiceKey": API_KEY,
            "applicant": company_name,
            "patent": "true",
            "utility": "true",
            "numOfRows": num_rows,
            "pageNo": page_no
        }

        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()

        data = xmltodict.parse(response.text)

        try:
            items = data["response"]["body"]["items"]["item"]
        except (KeyError, TypeError):
            break

        if isinstance(items, dict):
            items = [items]

        if not items:
            break

        all_items.extend(items)

    return all_items


In [ ]:
# !pip install tqdm

In [ ]:
company_df["이거쓰셈"].isna().sum()

In [ ]:
# =========================================================
# 5. 기업 리스트 순회 (전체 기업만)
# =========================================================
from tqdm import tqdm

all_results = []

company_list = company_df["이거쓰셈"].dropna().unique()

for company in tqdm(company_list, desc="기업별 특허 수집"):
    patents = fetch_patent_by_company(company)

    for p in patents:
        p_copy = p.copy()
        p_copy["company_name"] = company
        all_results.append(p_copy)


In [ ]:
# =========================================================
# 6. DataFrame 생성
# =========================================================
patent_df = pd.DataFrame(all_results)

print("특허 수집 대상 기업 수:", patent_df["company_name"].nunique())
print("전체 수집 특허 수:", len(patent_df))


In [ ]:
patent_df["company_name"].unique()

In [ ]:
# =========================================================
# 7. company_name / applicantName 정규화 컬럼 생성 (엄격 버전 사용)
# =========================================================
patent_df["company_norm"] = patent_df["company_name"].apply(normalize_name_strict)
patent_df["applicant_norm"] = patent_df["applicantName"].apply(normalize_name_strict)

# =========================================================
# 8. 동일 기업 특허만 필터링
    # matched_df 재생성 (완전일치만 허용)
# =========================================================
matched_df = patent_df[
    patent_df["company_norm"] == patent_df["applicant_norm"]
].copy()

# =========================================================
# 9. 결과 확인
# =========================================================
print("매칭된 기업 특허 수:", len(matched_df))
print("매칭된 기업 수:", matched_df["company_name"].nunique())


In [ ]:
# 엔켐 케이스 확인
matched_df[
    matched_df["company_name"].str.contains("엔켐", na=False)
][["company_name", "applicantName"]].drop_duplicates()

In [ ]:
matched_df = matched_df[
    [
        "ipcNumber",
        "applicationDate",
        "astrtCont",
        "applicationNumber",
        'registerDate',
        "indexNo",
        "registerStatus",
        "inventionTitle",
        "applicantName",
        "company_name"
                ]
]


In [ ]:
matched_df = matched_df.rename(columns={
    "ipcNumber": "ipcNumber_IPC코드",
    "applicationDate": "applicationDate_출원일자",
    "astrtCont": "astrtCont_초록",
    "applicationNumber": "applicationNumber_출원번호",
    "registerDate":"registerDate_등록일자",
    "indexNo": "indexNo_일련번호",
    "registerStatus": "registerStatus_등록상태",
    "inventionTitle": "inventionTitle_발명의명칭",
    "applicantName": "applicantName_출원인명",
    "company_name" : "company_name"
})


In [ ]:
matched_df['applicantName(출원인명)'].nunique()

In [ ]:
matched_df['company_name'].nunique()

In [ ]:
#====================================
# 10. CSV 로 저장 
#====================================
matched_df.to_csv("./preprocessed_data/company_patent_all_5.csv", index=False, encoding="utf-8-sig")


In [ ]:
#====================================
# 11. Excel 로 저장 
#====================================
matched_df.to_excel("./preprocessed_data/company_patent_all_5.xlsx",index=False)

# DART mapping 된 소부장 기업
- `dart_corp_code` 가 null 인 기업명은 날리기
- 기업명(`title`)로 특허 매핑 시키기

In [ ]:
import pandas as pd
#=====================

company_df = pd.read_csv('./data/소부장_기업_DART매핑_기업명정제.csv', encoding='cp949')
company_df.head()

In [ ]:
remove_nan = company_df[company_df['dart_corp_code'].notna()]
remove_nan.shape[0] # DART에 나와있는 기업은 총 478 개임

In [ ]:
remove_nan["title"].nunique()   # 469 개

In [ ]:
# title 기준 중복된 기업명만 추출
duplicate_titles = remove_nan[
    remove_nan["title"].duplicated(keep=False)
].sort_values("title")

duplicate_titles[["title", "dart_corp_code"]]

## 특허·실용 공개·등록공보(DART에 없는 기업은 제외)
- applicantName과 company_name 일치하게
- 앞뒤 공백 제거, 모든 공백 제거, 대소문자 통일

In [ ]:
import os
import requests
import xmltodict
import pandas as pd
from dotenv import load_dotenv
import re

#====================================
# 1. 기본 설정
#====================================
load_dotenv()

API_KEY = os.getenv("KIPRIS_API_KEY")

BASE_URL = (
    "http://plus.kipris.or.kr/kipo-api/"
    "kipi/patUtiModInfoSearchSevice/getAdvancedSearch"
)

#====================================
# 2. 기업명 정규화 함수 (공백, (주) 제거용)
#====================================
def normalize_name_for_match(name):
    if pd.isna(name):
        return ""
    name = name.lower()
    name = name.strip()
    name = re.sub(r"\s+", "", name)              # 모든 공백 제거
    name = re.sub(r"\(주\)|㈜|주식회사", "", name)
    return name

#====================================
# 3. 기업의 한 곳 특허 정보 가져오는 함수
#====================================
def fetch_patent_by_company(company_name, page_no=1, num_rows=50):
    params = {
        "ServiceKey": API_KEY,          
        "applicant": company_name,      # 검색 기준 기업명
        "patent": "true",
        "utility": "true",
        "numOfRows": num_rows,
        "pageNo": page_no
    }

    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()

    data = xmltodict.parse(response.text)

    try:
        items = data["response"]["body"]["items"]["item"]
    except (KeyError, TypeError):
        return []

    if isinstance(items, dict):
        items = [items]

    return items

#====================================
# 4. 기업명 전체 순회
#====================================
all_results = []

for company in remove_nan["title"].unique():
    print(f"수집 중: {company}")

    patents = fetch_patent_by_company(company)

    for p in patents:
        p["company_name"] = company
        all_results.append(p)

#====================================
# 5. DataFrame 생성
#====================================
patent_df = pd.DataFrame(all_results)

#====================================
# 6. company_name / applicantName 정규화 컬럼 추가
#====================================
patent_df["company_norm"] = patent_df["company_name"].apply(normalize_name_for_match)
patent_df["applicant_norm"] = patent_df["applicantName"].apply(normalize_name_for_match)

#====================================
# 7. 동일 기업 특허만 필터링
#====================================
matched_df = patent_df[
    patent_df["company_norm"] == patent_df["applicant_norm"]
].copy()

#====================================
# 8. 결과 확인
#====================================
print("전체 수집 특허 수:", len(patent_df))
print("동일 기업 특허 수:", len(matched_df))

matched_df[["company_name", "applicantName"]].head()


In [ ]:
matched_df.shape[0]

In [ ]:
#====================================
# 6. CSV 로 저장 
#====================================
matched_df.to_csv("./preprocessed_data/company_patent_raw_3.csv", index=False, encoding="utf-8-sig")


In [ ]:
#====================================
# 7. Excel 로 저장 
#====================================
matched_df.to_excel(
    "./preprocessed_data/company_patent_raw_3.xlsx",
    index=False
)


## dart nan 인 값 적용

In [ ]:
import os
import requests
import xmltodict
import pandas as pd
from dotenv import load_dotenv
import re

# =========================================================
# 1. 기본 설정
# =========================================================
load_dotenv()

API_KEY = os.getenv("KIPRIS_API_KEY")
if not API_KEY:
    raise RuntimeError("KIPRIS_API_KEY가 설정되어 있지 않습니다.")

BASE_URL = (
    "http://plus.kipris.or.kr/kipo-api/"
    "kipi/patUtiModInfoSearchSevice/getAdvancedSearch"
)

# =========================================================
# 2. DART 등록 기업만 필터링 (company_df 적용)
# =========================================================
company_df = pd.read_csv('./data/소부장_기업_DART매핑_기업명정제.csv', encoding='cp949')
remove_nan = company_df[company_df["dart_corp_code"].notna()].copy()    # dart_corp_code 가 있는 것들만 살려둠

print("DART 등록 기업 수:", remove_nan.shape[0])  # 478

# =========================================================
# 3. 기업명 정규화 함수
#    - 공백 제거
#    - (주), 주식회사, ㈜ 제거
#    - 대소문자 통일
# =========================================================
def normalize_name_for_match(name):
    if pd.isna(name):
        return ""
    name = name.lower()
    name = name.strip()
    name = re.sub(r"\s+", "", name)
    name = re.sub(r"\(주\)|㈜|주식회사", "", name)
    return name

# =========================================================
# 4. 기업 1곳 특허 정보 수집 함수 (KIPRIS)
# =========================================================
def fetch_patent_by_company(company_name, page_no=1, num_rows=50):
    params = {
        "ServiceKey": API_KEY,     
        "applicant": company_name,
        "patent": "true",
        "utility": "true",
        "numOfRows": num_rows,
        "pageNo": page_no
    }

    response = requests.get(BASE_URL, params=params, timeout=10)
    response.raise_for_status()

    data = xmltodict.parse(response.text)

    try:
        items = data["response"]["body"]["items"]["item"]
    except (KeyError, TypeError):
        return []

    if isinstance(items, dict):
        items = [items]

    return items

# =========================================================
# 5. 기업 리스트 순회 (DART 등록 기업만)
# =========================================================
all_results = []

for company in remove_nan["title"].unique():
    print(f"수집 중: {company}")

    patents = fetch_patent_by_company(company)

    for p in patents:
        p["company_name"] = company  # 검색 기준 기업명
        all_results.append(p)

# =========================================================
# 6. DataFrame 생성
# =========================================================
patent_df = pd.DataFrame(all_results)

print("특허 수집 대상 기업 수:", patent_df["company_name"].nunique())
print("전체 수집 특허 수:", len(patent_df))

# =========================================================
# 7. company_name / applicantName 정규화 컬럼 추가
# =========================================================
patent_df["company_norm"] = patent_df["company_name"].apply(normalize_name_for_match)
patent_df["applicant_norm"] = patent_df["applicantName"].apply(normalize_name_for_match)

# =========================================================
# 8. 동일 기업 특허만 필터링
#    (공백·회사형태 차이만 허용)
# =========================================================
matched_df = patent_df[
    patent_df["company_norm"] == patent_df["applicant_norm"]
].copy()

# =========================================================
# 9. 결과 확인
# =========================================================
print("동일 기업 특허 수:", len(matched_df))

matched_df[["company_name", "applicantName"]].head()


In [ ]:
matched_df["applicantName"].nunique()

In [ ]:
#====================================
# 6. CSV 로 저장 
#====================================
matched_df.to_csv("./preprocessed_data/company_patent_raw_4.csv", index=False, encoding="utf-8-sig")


In [ ]:
#====================================
# 7. Excel 로 저장 
#====================================
matched_df.to_excel("./preprocessed_data/company_patent_raw_4.xlsx",index=False)


In [ ]:
len(matched_df['company_name'].unique())

# 특허·실용 피인용문헌
- matched_df['applicationNumber']을 가지고 피인용정보 수집

In [ ]:
import os
import requests
import xmltodict
import pandas as pd
from dotenv import load_dotenv
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# =========================================================
# 1. 환경변수 로드
# =========================================================
load_dotenv()
API_KEY = os.getenv("KIPRIS_API_KEY")

if not API_KEY:
    raise RuntimeError("KIPRIS_API_KEY가 설정되어 있지 않습니다.")

# =========================================================
# 2. Endpoint (CitingService)
# =========================================================
CITING_URL = "http://plus.kipris.or.kr/openapi/rest/CitingService/citingInfo"

# =========================================================
# 3. Session + Retry 설정
# =========================================================
session = requests.Session()

retry_strategy = Retry(
    total=3,
    backoff_factor=1,
    status_forcelist=[500, 502, 503, 504],
    allowed_methods=["GET"]
)

adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)

In [ ]:
def fetch_citing_info(app_num):
    params = {
        "accessKey": API_KEY,
        "standardCitationApplicationNumber": app_num
    }

    try:
        res = session.get(CITING_URL, params=params, timeout=10)
        res.raise_for_status()
        raw = xmltodict.parse(res.text)
    except Exception:
        return None

    # 1️⃣ response 체크
    response = raw.get("response")
    if response is None:
        return None

    # 2️⃣ body 체크
    body = response.get("body")
    if body is None:
        return None

    # 3️⃣ items 체크
    items_wrapper = body.get("items")
    if items_wrapper is None:
        return None

    # 4️⃣ citingInfo 체크
    items = items_wrapper.get("citingInfo")
    if items is None:
        return None

    # 5️⃣ dict → list 보정
    if isinstance(items, dict):
        items = [items]

    df = pd.DataFrame(items)

    # 6️⃣ 컬럼명 표준화 (대소문자 안전)
    col_map = {c.lower(): c for c in df.columns}

    df = df.rename(columns={
        col_map.get("standardcitationapplicationnumber"): "StandardCitationApplicationNumber",
        col_map.get("applicationnumber"): "ApplicationNumber"
    })

    return df


In [ ]:
from tqdm import tqdm

# =========================================================
# 5. applicationNumber 기준으로 CitingService 순회
# =========================================================
citing_dfs = []

app_nums = (
    matched_df["applicationNumber_출원번호"]   
    .dropna()
    .astype(str)
    .unique()
)

for app_num in tqdm(app_nums, desc="📡 피인용 정보 수집 중"):
    citing_df = fetch_citing_info(app_num)

    if citing_df is not None and not citing_df.empty:
        citing_dfs.append(citing_df)

In [ ]:
# =========================================================
# 6. 피인용 결과 병합 (CitingService 결과 통합)
# =========================================================
if citing_dfs:
    citing_all_df = pd.concat(citing_dfs, ignore_index=True)
else:
    citing_all_df = pd.DataFrame(
        columns=["StandardCitationApplicationNumber", "ApplicationNumber"]
    )

# =========================================================
# 7. 기준 특허 → 기업명 + 출원인명 매핑 테이블 생성
# =========================================================
patent_company_map = (
    matched_df[[
        "applicationNumber_출원번호",
        "company_name",              # 정규화된 기업명 (분석용)
        "applicantName_출원인명"     # KIPRIS 원문 출원인명 (설명용)
    ]]
    .drop_duplicates()
)

# =========================================================
# 8. 기준 특허(StandardCitationApplicationNumber)에
#    company_name + applicantName 매핑
# =========================================================
final_df = citing_all_df.merge(
    patent_company_map,
    left_on="StandardCitationApplicationNumber",
    right_on="applicationNumber_출원번호",
    how="left"
)

# =========================================================
# 9. 관계 기준 중복 제거
# =========================================================
final_df = final_df.drop_duplicates(
    ["StandardCitationApplicationNumber", "ApplicationNumber"]
).reset_index(drop=True)

# =========================================================
# 10. 컬럼 정리
# =========================================================
final_df = final_df[[
    "StandardCitationApplicationNumber",  # 기준 특허
    "ApplicationNumber",                  # 인용한 특허
    "applicantName_출원인명",             # 기준 특허 원문 출원인명 (참고용)
    "company_name"                       # 기준 특허 소유 기업 (분석 기준)
]]

# =========================================================
# 11. 결과 확인
# =========================================================
print(f"✅ 최종 피인용 관계 수: {len(final_df)}")
print("✅ 기준 특허 수:", final_df["StandardCitationApplicationNumber"].nunique())
print("✅ 기업 수:", final_df["company_name"].nunique())

final_df.head()


In [ ]:
#====================================
# 10. CSV 로 저장 
#====================================
final_df.to_csv("./preprocessed_data/company_patent_citing_6.csv", index=False, encoding="utf-8-sig")


In [ ]:
#====================================
# 11. Excel 로 저장 
#====================================
final_df.to_excel("./preprocessed_data/company_patent_citing_6.xlsx",index=False)

# 피인용 데이터 전처리
- 공동출원 기업 관련해서 수기로 작업하지 않기 위해 1차 전처리